In [2]:
import re
from nltk.tokenize import TweetTokenizer
import pandas as pd
from ast import literal_eval
import numpy as np
import os


In [3]:
df_all_tweets = pd.read_csv('../team_data/all_tweets.csv', index_col=0)

### 18 seconds for 100 tweets

## Full Classification Example
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest?text=Covid+cases+are+increasing+fast%21

In [ ]:
texts = [
    "Covid cases are increasing fast! http://www.google.ca", 
    "How bout them bears @bears, go bears!", 
    "packers stink, no good",
    "something happened",
    " I like stuff, how about you ?"
    ]
text = "Covid cases are increasing fast! http://www.google.ca"



In [4]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)

def preprocess_single_text(text):
  new_text = []
  for t in text.split(" "):
      t = '@user' if t.startswith('@') and len(t) > 1 else t
      t = 'http' if t.startswith('http') else t
      new_text.append(t)
  return " ".join(new_text)

def preprocess_tweets(df):
  df['tweet_text'] = df['tweet_text'].map(lambda x: preprocess_single_text(x))
  return df


df_all_tweets = preprocess_tweets(df_all_tweets)

In [5]:
datasets_path = '/content/drive/My Drive/DATA SCIENCE MSC/DSM500 - Final Project/Datasets'

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)



Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def get_labels(scores):
  results = []
  for score in scores:
    # get softmax scores
    score = softmax(score)
    # get rankings
    ranking = np.argsort(score)[::-1]
    # get top softmax score
    top_score = score[ranking[0]]
    # get top label
    top_label = config.id2label[ranking[0]]
    # print(top_score, top_label)
    results.append({'label': top_label, 'score': top_score})
  return results


# labels = get_labels(scores)

In [7]:
def get_sentiment_2(df, n_splits):
    # split df into chunks
    split_dfs = np.array_split(df, n_splits)
    
    for i , df in enumerate(split_dfs[:1]):
      # preprocess tweets
      # df = preprocess_tweets(df)
    
      # convert to list
      processed_tweets = df['tweet_text'].to_list()
      # encode inputs
      encoded_inputs = tokenizer(processed_tweets, padding=True, truncation=True, max_length = 512, return_tensors='pt')
      # get outputs
      outputs = model(**encoded_inputs)
      # get scores
      scores = outputs[0].detach().numpy()
      # get labels
      labels = get_labels(scores)
      
      # create dataframe with sentiment
      sentiment_df = pd.DataFrame.from_dict(labels)
      # concat sentiment df with tweet df and return
      df_tweets_w_sentiment = pd.concat([df, sentiment_df], axis=1)
      df_tweets_w_sentiment.to_csv(f'../team_data/all_tweets_sentiment{i}.csv')
      # return df_tweets_w_sentiment

In [10]:
# df_some_tweets = df_all_tweets.iloc[:1000].copy()



get_sentiment_2(df_all_tweets, 100000)

In [11]:

# split_dfs = np.array_split(df_all_tweets, 10000)
# df1 = split_dfs[0]
# list_tweets = df1['tweet_text'].to_list()
# encoded_inputs = tokenizer(list_tweets, padding=True, truncation=True, max_length = 128, return_tensors='pt')
# # get outputs
# outputs = model(**encoded_inputs)
# # # get scores
# scores = outputs[0].detach().numpy()
# # # get labels
# labels = get_labels(scores)


## Fine Tune Train model
would require labeling some data - not going to do I don't think

In [ ]:
language_model = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
task = "sentiment"

# load dataset
dataset, label_to_id = tweetnlp.load_dataset(task)

# load trainer
trainer_class = tweetnlp.load_trainer(task)

# define trainer
trainer = trainer_class(
    language_model=language_model,
    dataset=dataset,
    label_to_id=label_to_id,
    max_length=128,
    split_train='train',
    split_test='test',
    output_dir=f'model_ckpt/test'
)

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to C:/Users/hugho/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


...

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

c:\Users\hugho\AppData\Local\Programs\Python\Python310\lib\site-packages\tweetnlp\text_classification\trainer.py:94: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_accuracy = load_metric("accuracy")


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45615
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12284
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [ ]:
# train
trainer.train(down_sample_size_train=1000, ray_result_dir="ray_results/test")

# save model checkpoint
trainer.save_model()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\hugho\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 45615
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 17106


  0%|          | 0/17106 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
text = "Covid cases are increasing fast!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)



Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# scores

In [ ]:
ranking = np.argsort(scores)
ranking = ranking[::-1]

In [ ]:


# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores

for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


1) negative 0.7236
2) neutral 0.2287
3) positive 0.0477
